# $\textbf{Part 1. Predicting LFSR States Using Feedforward Neural Networks}$
$\text{Author: Ryan Burns}$

#### $\textbf{Linear Feedback Shift Registers (LFSRs)}$
$\text{For background on LFSRs and maximum-length }m\text{-sequences, visit: }\textit{https://en.wikipedia.org/wiki/Linear-feedback_shift_register.}$
$\text{It is assumed that a finite list of primitive polynomial coefficients (stored as hex values) are available for the polynomial degree specified below}$
$\text{via a locally stored file with name <degree>.txt. For degree-10 polynomials, for example, the corresponding primitive polynomial coefficients}$
$\text{would have file name 10.txt. LFSRs are built for each polynomial, each yielding a pseudorandom binary sequence of length/period }2^{\text{degree}}-1.$

#### $\textbf{Prediction of LFSR State }n + 1\textbf{ Given State }n$
$\text{Suppose we are given an arbitrary LFSR feedback polynomial and we build a representation of the LFSR in software. Using this LFSR, and}$
$\text{assuming the feedback polynomial is primitive in a polynomial ring of fixed degree over }GF(2)\text{, we can generate a maximum-length binary}$
$\text{sequence which has well-documented correlation properties which make them desirable building blocks for stream ciphers or spread-spectrum}$
$\text{codes, to name a few examples. On their own, LFSRs do not afford any real cryptographic protection, being famously susceptible to correlation}$
$\text{attacks. Speaking of attacks, our goal in this notebook is to demonstrate that we can use a }\textit{very}\text{ simple neural network to predict LFSR behavior.}$
$\text{Specifically, we posit that a two-layer (i.e., single hidden-layer) feedforward neural network can readily predict the state-evolution of an arbitrary}$
$\text{LFSR. For simplicity, we constrain this network to have a very specific architecture. Its input layer consists of a fixed-length binary vector that}$
$\text{represents the LFSR state vector at some time index (epoch) }n\text{. We use the ground truth register state at index / epoch }n + 1\text{ to train a the network}$
$\text{to develop a latent representation of the finite field recursion. Both layers of the network are sigmoid connections, representing soft approximations}$
$\text{of boolean gates, outputing activations in the unit interval, }[0,1].\text{ These layers assume no bias terms, only weighted combinations of inputs. For}$
$\text{an LFSR of recursion-order/polynomial degree }deg\text{, the first (and only) hidden layer consists a total of }2\times deg\text{ weighted sigmoidal activations.}$
$\text{The output layer consists of the }deg\text{ total activations predicting }deg\text{ total bits in the LFSR state vector at epoch }n + 1\text{, for all }n\text{. These binary}$
$\text{activations are trained against the true LFSR state vectors at these indices }n + 1\text{ using a binary cross-entropy error function. The population of}$
$\text{observations are the }deg\text{-length LFSR states across all epochs in one full }m\text{-sequence period / cycle. The labels for the error function are the }$
$deg\text{-length state vector values upon 1 iteration of the LFSR recursion, per observation.}$

#### $\textbf{Part 1 of This Notebook}$
$\text{Part 1 of this notebook prototypes a feedforward binary (bias-less) neural network using Keras, training it as described above using RMSprop. The}$
$\text{model is given too few degrees of freedom to completely memorize the bits, so it must learn a latent representation of the underlying finite field}$
$\text{mathematics defining the recurrence relation. After sufficiently many training epochs, the model has been tested to converge to 100}\%\text{ accuracy}$
$\text{for all degree-10 primitive polynomials in the file 10.txt. Primitive polynomials defining }m\text{-sequences for other polynomial degrees are available}$
$\text{for download at the following site: }\textit{https://users.ece.cmu.edu/~koopman/lfsr/index.html}\text{. Future efforts may build this proof-of-concept out to}$
$\text{higher degree recursions and more complicated kinds of stream ciphers. For now, we demonstrate that a relatively shallow binary network is}$
$\text{sufficiently powerful to "crack" (i.e., predict) an LFSR's underlying recursive state vector mapping.}$

$\text{In addition to training the network, we animate the results in a plot which depicts the time-evolution of the LFSR recursion and the neural}$
$\text{network's state predictions. In addition, options are provided for plotting the correlation properties of the }m\text{-sequence training data and the Keras}$
$\text{model metrics vs training epoch, respectively. Documentation and source code for all the functions below can be found in this repo's modules.}$

### $\textbf{Import Dependencies}$

In [1]:
# System imports
from os import environ
from time import time

# So numpy and Tensorflow can cooperate
environ['KMP_DUPLICATE_LIB_OK']='True'

# Numpy imports
from numpy import array, matmul, transpose, identity, hstack, vstack, shape
from numpy import expand_dims, zeros, correlate, linspace, arange, max

# Galois / LFSR utilities
from galois_tools import *
from m_sequence_viz import *
from lfsr_network_viz import *
from ml_utils import *

# Plotting functionality
from matplotlib import pyplot as plt
from matplotlib.animation import FuncAnimation

# Tensorflow/Keras functionality
from tensorflow.keras.layers import Input, Dense
from tensorflow.keras.models import Model;
from tensorflow.keras.optimizers import RMSprop;
from tensorflow.keras.metrics import TruePositives, FalsePositives, TrueNegatives, FalseNegatives;

### $\textbf{Plotting Macro (Requires IPython)}$

In [2]:
%matplotlib notebook

### $\textbf{Specify LFSR & Neural Net Parameters}$

In [3]:
###########################
# Specify LFSR parameters #
###########################

# Primitive polynomial degree
deg = 10 # (= number of bits)

# Length of m-sequence generated
M = 2**deg - 1 # (bits)

# Initial state of register
seed = 1 # (decimal form)

# Primitive polynomial choice
p = 16 # (index in catalog)

###############################
# Network training parameters #
###############################

# Number of RMSprop training epochs
N_epoch = 16000

# Batch size equal to m-sequence length
N_batch = 2**deg - 1

# Learning rate for RMSprop
learning_rate = 0.001

#########################
# Behavioral parameters #
#########################

# Turn ON/OFF correlation plot
plot_correlations = True

# Turn ON/OFF true vs. predicted mask plot
plot_prediction_masks = True

### $\textbf{Load Primitive Polynomial Coefficients of Specified Degree Over }GF(2)$

In [4]:
#############################################################
# Load coeff. of primitive polynomials over GF(2) from file #
#############################################################

# Read list of primitive polynomial coefficients for definition
# of the linear feedback shift registers yielding m-sequences
coeff_catalog = read_polynomials_from_file(deg=deg,
    path='./primitive_polynomials/')

###########################################################
# Generate linear feedback shift registers w/ polynomials #
###########################################################

# Define a bank of LFSRs - 1 per polynomial in the catalog
LFSRs = [LFSR(mask=eval(hex2bin(coeff_catalog[n], nbits=deg)), 
    seed=seed, order=deg, register_id=str(n)) 
    for n in range(len(coeff_catalog))]

### $\textbf{Visualize Auto- & Cross-Correlations of Two m-Sequences}$

In [5]:
# Optional correlation plot...
if plot_correlations:

    ##################################################
    # Generate m-sequences for each LFSR in the bank #
    ##################################################

    # Maximimum length binary sequences of length M = 2^deg - 1 via LFSRs
    m_sequences = vstack(tuple(register.m_sequence() for register in LFSRs))

    # Convert m-sequences from logical {0,1} to algebraic {-1,+1} representation
    algebraic_m_sequences = m_sequences.astype('float64') * 2 - 1

    ###################################################
    # Visualize correlation properties of m-sequences #
    ###################################################

    # Plot the correlations of the 1st 2 m-sequences generated above
    correlation_example(m_sequence0=algebraic_m_sequences[p,:], 
        m_sequence1=algebraic_m_sequences[p + 1,:],  deg=deg)

<IPython.core.display.Javascript object>

### $\textbf{Form Observations of }n^{\text{th}}\textbf{ LFSR State for Prediction of }(n + 1)^\text{st}\textbf{ State, for }n=0,1,2,\ldots$

In [6]:
#########################################################
# Buffer in b[n] a total of 2^deg + deg m-sequence bits #
#########################################################

# Maximimum length binary sequences of length M = 2^deg - 1 via LFSRs
b = vstack(tuple(register.stream(2**deg + deg) for register in LFSRs))

########################################################
# n'th LFSR register state observations, n = 0,1,2,... #
########################################################

# Input LFSR windows (deg-bit observation vectors)
X = array([b[p,n:(n + deg)] for n in range(b.shape[1] - deg - 1)])

############################################################
# (n+1)'st LFSR register state observations, n = 0,1,2,... #
############################################################

# Target LFSR windows, 1 epoch into future from X
Y = array([b[p,(n + 1):(1 + n + deg)] for n in range(b.shape[1] - deg - 1)])

### $\textbf{Define Feedforward Binary Neural Network}$

In [7]:
# Use Keras to define feedforward binary neural network, with
# two sigmoidal layers (without bias terms) for estimation of
# the LFSR state vector at epoch n + 1 from the state at epoch n
model = feedforward_lfsr_predictor(deg=deg, 
    learning_rate=learning_rate, print_summary=True)

Model: "model"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
input (InputLayer)           [(None, 10)]              0         
_________________________________________________________________
hidden (Dense)               (None, 20)                200       
_________________________________________________________________
output (Dense)               (None, 10)                200       
Total params: 400
Trainable params: 400
Non-trainable params: 0
_________________________________________________________________


### $\textbf{Fit Sigmoid Model to Predict Register State }n + 1\textbf{ from State }n$

In [8]:
# Model training
model.fit(
    
    # Input dataset
    x=X, # state vectot "observation" bits
    y=Y, # future-state predicted bits
    
    # Batch size
    batch_size=N_batch,
    
    # Number of training epochs
    epochs=N_epoch,
    
    # Print progress
    verbose=1,
    
    # Set aside fraction for validation
    validation_split=0, # (use everything)
    
    # False for time series
    shuffle=False,
    
    # Parallelize job across 2 workers
    workers=1,
    use_multiprocessing=False);

Train on 1023 samples
Epoch 1/16000
1023/1023 [==============================] - 1s 1ms/sample - loss: 0.7127 - true_positives: 2275.0000 - true_negatives: 2920.0000 - false_positives: 2190.0000 - false_negatives: 2845.0000
Epoch 2/16000
1023/1023 [==============================] - 0s 6us/sample - loss: 0.7090 - true_positives: 2265.0000 - true_negatives: 2974.0000 - false_positives: 2136.0000 - false_negatives: 2855.0000
Epoch 3/16000
1023/1023 [==============================] - 0s 5us/sample - loss: 0.7065 - true_positives: 2274.0000 - true_negatives: 2982.0000 - false_positives: 2128.0000 - false_negatives: 2846.0000
Epoch 4/16000
1023/1023 [==============================] - 0s 5us/sample - loss: 0.7046 - true_positives: 2294.0000 - true_negatives: 2998.0000 - false_positives: 2112.0000 - false_negatives: 2826.0000
Epoch 5/16000
1023/1023 [==============================] - 0s 5us/sample - loss: 0.7029 - true_positives: 2310.0000 - true_negatives: 3019.0000 - false_positives: 2091.00

Epoch 42/16000
1023/1023 [==============================] - 0s 5us/sample - loss: 0.6753 - true_positives: 3054.0000 - true_negatives: 3302.0000 - false_positives: 1808.0000 - false_negatives: 2066.0000
Epoch 43/16000
1023/1023 [==============================] - 0s 5us/sample - loss: 0.6748 - true_positives: 3078.0000 - true_negatives: 3315.0000 - false_positives: 1795.0000 - false_negatives: 2042.0000
Epoch 44/16000
1023/1023 [==============================] - 0s 5us/sample - loss: 0.6743 - true_positives: 3090.0000 - true_negatives: 3315.0000 - false_positives: 1795.0000 - false_negatives: 2030.0000
Epoch 45/16000
1023/1023 [==============================] - 0s 5us/sample - loss: 0.6738 - true_positives: 3108.0000 - true_negatives: 3320.0000 - false_positives: 1790.0000 - false_negatives: 2012.0000
Epoch 46/16000
1023/1023 [==============================] - 0s 5us/sample - loss: 0.6733 - true_positives: 3125.0000 - true_negatives: 3333.0000 - false_positives: 1777.0000 - false_negati

Epoch 83/16000
1023/1023 [==============================] - 0s 6us/sample - loss: 0.6559 - true_positives: 3665.0000 - true_negatives: 3712.0000 - false_positives: 1398.0000 - false_negatives: 1455.0000
Epoch 84/16000
1023/1023 [==============================] - 0s 7us/sample - loss: 0.6554 - true_positives: 3676.0000 - true_negatives: 3722.0000 - false_positives: 1388.0000 - false_negatives: 1444.0000
Epoch 85/16000
1023/1023 [==============================] - 0s 7us/sample - loss: 0.6549 - true_positives: 3691.0000 - true_negatives: 3729.0000 - false_positives: 1381.0000 - false_negatives: 1429.0000
Epoch 86/16000
1023/1023 [==============================] - 0s 7us/sample - loss: 0.6545 - true_positives: 3697.0000 - true_negatives: 3737.0000 - false_positives: 1373.0000 - false_negatives: 1423.0000
Epoch 87/16000
1023/1023 [==============================] - 0s 6us/sample - loss: 0.6540 - true_positives: 3714.0000 - true_negatives: 3746.0000 - false_positives: 1364.0000 - false_negati

Epoch 124/16000
1023/1023 [==============================] - 0s 6us/sample - loss: 0.6369 - true_positives: 4235.0000 - true_negatives: 3968.0000 - false_positives: 1142.0000 - false_negatives: 885.0000
Epoch 125/16000
1023/1023 [==============================] - 0s 5us/sample - loss: 0.6365 - true_positives: 4243.0000 - true_negatives: 3979.0000 - false_positives: 1131.0000 - false_negatives: 877.0000
Epoch 126/16000
1023/1023 [==============================] - 0s 5us/sample - loss: 0.6360 - true_positives: 4250.0000 - true_negatives: 3988.0000 - false_positives: 1122.0000 - false_negatives: 870.0000
Epoch 127/16000
1023/1023 [==============================] - 0s 5us/sample - loss: 0.6355 - true_positives: 4264.0000 - true_negatives: 3986.0000 - false_positives: 1124.0000 - false_negatives: 856.0000
Epoch 128/16000
1023/1023 [==============================] - 0s 5us/sample - loss: 0.6351 - true_positives: 4257.0000 - true_negatives: 4013.0000 - false_positives: 1097.0000 - false_negat

Epoch 165/16000
1023/1023 [==============================] - 0s 6us/sample - loss: 0.6176 - true_positives: 4482.0000 - true_negatives: 4132.0000 - false_positives: 978.0000 - false_negatives: 638.0000
Epoch 166/16000
1023/1023 [==============================] - 0s 6us/sample - loss: 0.6171 - true_positives: 4458.0000 - true_negatives: 4159.0000 - false_positives: 951.0000 - false_negatives: 662.0000
Epoch 167/16000
1023/1023 [==============================] - 0s 6us/sample - loss: 0.6166 - true_positives: 4484.0000 - true_negatives: 4150.0000 - false_positives: 960.0000 - false_negatives: 636.0000
Epoch 168/16000
1023/1023 [==============================] - 0s 7us/sample - loss: 0.6162 - true_positives: 4464.0000 - true_negatives: 4168.0000 - false_positives: 942.0000 - false_negatives: 656.0000
Epoch 169/16000
1023/1023 [==============================] - 0s 6us/sample - loss: 0.6157 - true_positives: 4491.0000 - true_negatives: 4155.0000 - false_positives: 955.0000 - false_negatives:

...

Epoch 15817/16000
1023/1023 [==============================] - 0s 7us/sample - loss: 1.4463e-05 - true_positives: 5120.0000 - true_negatives: 5110.0000 - false_positives: 0.0000e+00 - false_negatives: 0.0000e+00
Epoch 15818/16000
1023/1023 [==============================] - 0s 6us/sample - loss: 1.4456e-05 - true_positives: 5120.0000 - true_negatives: 5110.0000 - false_positives: 0.0000e+00 - false_negatives: 0.0000e+00
Epoch 15819/16000
1023/1023 [==============================] - 0s 6us/sample - loss: 1.4448e-05 - true_positives: 5120.0000 - true_negatives: 5110.0000 - false_positives: 0.0000e+00 - false_negatives: 0.0000e+00
Epoch 15820/16000
1023/1023 [==============================] - 0s 7us/sample - loss: 1.4441e-05 - true_positives: 5120.0000 - true_negatives: 5110.0000 - false_positives: 0.0000e+00 - false_negatives: 0.0000e+00
Epoch 15821/16000
1023/1023 [==============================] - 0s 6us/sample - loss: 1.4434e-05 - true_positives: 5120.0000 - true_negatives: 5110.0000 

Epoch 15856/16000
1023/1023 [==============================] - 0s 7us/sample - loss: 1.4185e-05 - true_positives: 5120.0000 - true_negatives: 5110.0000 - false_positives: 0.0000e+00 - false_negatives: 0.0000e+00
Epoch 15857/16000
1023/1023 [==============================] - 0s 7us/sample - loss: 1.4178e-05 - true_positives: 5120.0000 - true_negatives: 5110.0000 - false_positives: 0.0000e+00 - false_negatives: 0.0000e+00
Epoch 15858/16000
1023/1023 [==============================] - 0s 7us/sample - loss: 1.4171e-05 - true_positives: 5120.0000 - true_negatives: 5110.0000 - false_positives: 0.0000e+00 - false_negatives: 0.0000e+00
Epoch 15859/16000
1023/1023 [==============================] - 0s 6us/sample - loss: 1.4164e-05 - true_positives: 5120.0000 - true_negatives: 5110.0000 - false_positives: 0.0000e+00 - false_negatives: 0.0000e+00
Epoch 15860/16000
1023/1023 [==============================] - 0s 6us/sample - loss: 1.4157e-05 - true_positives: 5120.0000 - true_negatives: 5110.0000 

Epoch 15895/16000
1023/1023 [==============================] - 0s 6us/sample - loss: 1.3916e-05 - true_positives: 5120.0000 - true_negatives: 5110.0000 - false_positives: 0.0000e+00 - false_negatives: 0.0000e+00
Epoch 15896/16000
1023/1023 [==============================] - 0s 5us/sample - loss: 1.3909e-05 - true_positives: 5120.0000 - true_negatives: 5110.0000 - false_positives: 0.0000e+00 - false_negatives: 0.0000e+00
Epoch 15897/16000
1023/1023 [==============================] - 0s 5us/sample - loss: 1.3902e-05 - true_positives: 5120.0000 - true_negatives: 5110.0000 - false_positives: 0.0000e+00 - false_negatives: 0.0000e+00
Epoch 15898/16000
1023/1023 [==============================] - 0s 5us/sample - loss: 1.3895e-05 - true_positives: 5120.0000 - true_negatives: 5110.0000 - false_positives: 0.0000e+00 - false_negatives: 0.0000e+00
Epoch 15899/16000
1023/1023 [==============================] - 0s 5us/sample - loss: 1.3888e-05 - true_positives: 5120.0000 - true_negatives: 5110.0000 

Epoch 15934/16000
1023/1023 [==============================] - 0s 6us/sample - loss: 1.3653e-05 - true_positives: 5120.0000 - true_negatives: 5110.0000 - false_positives: 0.0000e+00 - false_negatives: 0.0000e+00
Epoch 15935/16000
1023/1023 [==============================] - 0s 7us/sample - loss: 1.3647e-05 - true_positives: 5120.0000 - true_negatives: 5110.0000 - false_positives: 0.0000e+00 - false_negatives: 0.0000e+00
Epoch 15936/16000
1023/1023 [==============================] - 0s 7us/sample - loss: 1.3640e-05 - true_positives: 5120.0000 - true_negatives: 5110.0000 - false_positives: 0.0000e+00 - false_negatives: 0.0000e+00
Epoch 15937/16000
1023/1023 [==============================] - 0s 7us/sample - loss: 1.3633e-05 - true_positives: 5120.0000 - true_negatives: 5110.0000 - false_positives: 0.0000e+00 - false_negatives: 0.0000e+00
Epoch 15938/16000
1023/1023 [==============================] - 0s 7us/sample - loss: 1.3627e-05 - true_positives: 5120.0000 - true_negatives: 5110.0000 

Epoch 15973/16000
1023/1023 [==============================] - 0s 6us/sample - loss: 1.3398e-05 - true_positives: 5120.0000 - true_negatives: 5110.0000 - false_positives: 0.0000e+00 - false_negatives: 0.0000e+00
Epoch 15974/16000
1023/1023 [==============================] - 0s 6us/sample - loss: 1.3392e-05 - true_positives: 5120.0000 - true_negatives: 5110.0000 - false_positives: 0.0000e+00 - false_negatives: 0.0000e+00
Epoch 15975/16000
1023/1023 [==============================] - 0s 6us/sample - loss: 1.3385e-05 - true_positives: 5120.0000 - true_negatives: 5110.0000 - false_positives: 0.0000e+00 - false_negatives: 0.0000e+00
Epoch 15976/16000
1023/1023 [==============================] - 0s 6us/sample - loss: 1.3379e-05 - true_positives: 5120.0000 - true_negatives: 5110.0000 - false_positives: 0.0000e+00 - false_negatives: 0.0000e+00
Epoch 15977/16000
1023/1023 [==============================] - 0s 6us/sample - loss: 1.3372e-05 - true_positives: 5120.0000 - true_negatives: 5110.0000 

### $\textbf{Plot LFSR State Observation & Network's Future State Predictions}$

In [9]:
# Plot true n'th vs. predicted (n + 1)'st'...
if plot_prediction_masks:
    
    # Plot LFSR state progression & predicted progression (in parallel)
    true_vs_predicted(idx0=0, idx1=50, input_obs=X,
                      output_activations=output_activations)

NameError: name 'true_vs_predicted' is not defined

### $\textbf{Get Activations of Hidden & Output Layers}$

In [ ]:
######################################
# Hidden sigmoidal layer activations #
######################################

# Gather hidden layer outputs from original model
h_layer = model.get_layer('hidden').output

# Separate model to spy on hidden layer activations
hidden_layer_model = Model(inputs=model.input,
    outputs=h_layer)

# Sigmoidal hidden layer outputs / activations
hidden_activations = hidden_layer_model.predict(X)

######################################
# Output sigmoidal layer activations #
######################################

# Get model output activations
output_activations = model.predict(X)

### $\textbf{Visualize LFSR State and Feedforward Binary Network}$

In [ ]:
##############################
# Visualization config panel #
##############################

config = {
    'figsize': (9.9, 7),
    'deg': deg,
    'x_mid': (deg - 1) / 2,
    'cmap': 'cividis',
    'y_input': 0,
    'y_hidden': 0.45,
    'y_output': 0.9,
    'y_decision': 1,
    'y_LFSR_loop': -0.1,
    'ylim': [-0.32, 1.2],
    'num_hidden': 2 * deg,
    'num_input': deg,
    'num_output': deg,
    'node_size': 10,
    'link_width': 1,
    'net_link_alpha': 0.6,
    'arrow_size': 8,
    'net_feedback_linestyle':'--'
}

lfsr_prediction_animation(
    lfsr=LFSRs[p], config=config, model=model,
    state_sequence=X, hidden_activations=hidden_activations,
    output_activations=output_activations,
    run_animation=True, interval=1000, blit=True, repeat=True)